In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import torch
import random
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
import logging
import cv2

cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

logging.getLogger('albumentations').setLevel(logging.WARNING)
from albumentations.pytorch import ToTensorV2

from rsna2024.runner import Runner
from rsna2024.preproc.generate_tiles import get_tile
from rsna2024 import model as module_model
from rsna2024.utils import (
    rsna_lumbar_metric,
    natural_sort,
    sagi_coord_to_axi_instance_number,
    get_series,
)

root_dir = '/media/latlab/MR/projects/kaggle-rsna-2024'
data_dir = os.path.join(root_dir, 'data', 'raw')
img_dir = os.path.join(data_dir, 'train_images')
df_series = pd.read_csv(
    os.path.join(data_dir, 'train_series_descriptions.csv'),
    dtype={'study_id': 'str', 'series_id': 'str'},
)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

levels = ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']
sides = ['left', 'right']
coord_df = pd.read_csv(
    os.path.join(root_dir, 'data', 'processed', 'train_label_coordinates_predicted.csv')
)


def load_config(config_path):
    with open(config_path) as f:
        return json.load(f)


def get_metric(y_true, y_pred):
    solution, submission = rsna_lumbar_metric.prepare_data(y_true, y_pred)
    metric = rsna_lumbar_metric.score(
        solution=solution,
        submission=submission,
        row_id_column_name='row_id',
        any_severe_scalar=1.0,
    )
    return metric

def print_loss(orig_baseline_preds, baseline_preds, baseline_ys, idx):
    orig_baseline_loss = CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))(orig_baseline_preds[..., idx], baseline_ys[..., idx])
    baseline_loss = CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))(baseline_preds[..., idx], baseline_ys[..., idx])
    print(f'\nLoss: {orig_baseline_loss:.4f} -> {baseline_loss:.4f}')
    print(f'Improvement: {orig_baseline_loss - baseline_loss:.4f}, {100 * (orig_baseline_loss - baseline_loss) / orig_baseline_loss:.2f}%')

## Baseline model

In [ ]:
model_name = 'comfy-galaxy-426'
baseline_cfg = load_config(
    os.path.join(root_dir, 'models', 'rsna-2024-' + model_name, 'config.json')
)
baseline_preds, baseline_ys, data = Runner(
    baseline_cfg, model_name='rsna-2024-' + model_name
).predict()

baseline_preds = torch.unflatten(torch.tensor(baseline_preds).to(device), 1, [3, -1])
orig_baseline_preds = baseline_preds.clone()
baseline_ys = torch.tensor(baseline_ys).to(device)
get_metric(baseline_ys, baseline_preds)

## ROI Models
### Spinal

In [ ]:
spinal_model_name = 'fine-field-600'
cfg = load_config(os.path.join(root_dir, 'models', 'rsna-2024-' + spinal_model_name, 'config.json'))
spinal_preds, spinal_ys, spinal_data = Runner(
    cfg, model_name='rsna-2024-' + spinal_model_name
).predict(df_coordinates=coord_df)

# unfolding 5 cv folds and 5 levels
spinal_preds = np.moveaxis(spinal_preds.reshape(5, 5, -1, 3), 1, -1).reshape(-1, 3, 5)

spinal_idx = np.arange(5)
baseline_preds[..., spinal_idx] = torch.tensor(spinal_preds).to(device)
print_loss(orig_baseline_preds, baseline_preds, baseline_ys, spinal_idx)

### Foraminal

In [ ]:
foraminal_model_name = 'devoted-pine-604'
cfg = load_config(os.path.join(root_dir, 'models', 'rsna-2024-' + foraminal_model_name, 'config.json'))
foraminal_preds, foraminal_ys, foraminal_data = Runner(
    cfg, model_name='rsna-2024-' + foraminal_model_name
).predict(df_coordinates=coord_df)

# unfolding 5 cv folds, 5 levels, (study_ids) and 2 sides
foraminal_preds = np.moveaxis(np.moveaxis(foraminal_preds.reshape(5, 5, -1, 2, 3), 3, -1), 1, -1).reshape(-1, 3, 10)

foraminal_idx = np.arange(5, 15)
baseline_preds[..., foraminal_idx] = torch.tensor(foraminal_preds).to(device)
print_loss(orig_baseline_preds, baseline_preds, baseline_ys, foraminal_idx)

### Subarticular

In [ ]:
subarticular_model_name = 'still-dragon-595'
cfg = load_config(os.path.join(root_dir, 'models', 'rsna-2024-' + subarticular_model_name, 'config.json'))
subarticular_preds, subarticular_ys, subarticular_data = Runner(
    cfg, model_name='rsna-2024-' + subarticular_model_name
).predict(df_coordinates=coord_df)

# unfolding 5 cv folds, 5 levels and 2 sides
subarticular_preds = np.moveaxis(np.moveaxis(subarticular_preds.reshape(5, 5, -1, 2, 3), 3, -1), 1, -1).reshape(-1, 3, 10)

subarticular_idx = np.arange(15, 25)
baseline_preds[..., subarticular_idx] = torch.tensor(subarticular_preds).to(device)
print_loss(orig_baseline_preds, baseline_preds, baseline_ys, subarticular_idx)

### Global improvement

In [ ]:
print_loss(orig_baseline_preds, baseline_preds, baseline_ys, np.arange(25))

orig_baseline_metric = get_metric(baseline_ys, orig_baseline_preds)
baseline_metric = get_metric(baseline_ys, baseline_preds)
print(f'Baseline metric: {orig_baseline_metric:.4f} -> {baseline_metric:.4f}')
print(f'Improvement: {orig_baseline_metric - baseline_metric:.4f}, {100 * (orig_baseline_metric - baseline_metric) / orig_baseline_metric:.2f}%')